In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import matplotlib.patches as patches

xlen = 253
ylen = 235
xsize = 0.01175 / xlen
ysize = 0.01095 / ylen

hwDir = 'hw_heatmaps'
simDir = 'outputs'
Rodinia = ['backprop','bfs','b+tree','dwt2d','gaussian',
           'heartwall','hotspot', 'kmeans','lavaMD',
           'lud','myocyte','nw']

hw_avg = pd.DataFrame(np.zeros((235,253)))
sim_avg = pd.DataFrame(np.zeros((235,253)))
    
hw_avg /= len(Rodinia)
sim_avg /= len(Rodinia)

#load in heatmap
simHeatmaps = {}
sim_avg =pd.DataFrame(np.zeros((235,253)))
for bm in Rodinia:
    filename = os.path.join(simDir,(bm + '.grid.steady'))
    fp = open(filename, "r")
    temp = []
    found = False
    for line in fp:
        if found:
            temp.append(float(line.strip().split()[1]))
        else:
            if line.strip() == "Layer 8:":
                found = True
    temp = np.reshape(temp,(256,256))
    temp = pd.DataFrame(np.array(Image.fromarray(temp).resize((253,235))))
    temp -= 273.15
    simHeatmaps[bm] = temp



In [6]:
simHeatmaps

{'backprop':             0           1           2           3           4           5    \
 0    132.131989  133.611115  134.750488  135.643646  136.326447  136.874756   
 1    133.805878  135.308197  136.476959  137.382812  138.093475  138.645081   
 2    135.071625  136.605316  137.786896  138.712921  139.425415  139.994629   
 3    136.011871  137.564331  138.768677  139.701904  140.425598  140.997589   
 4    136.720856  138.283875  139.506714  140.443329  141.178650  141.757721   
 ..          ...         ...         ...         ...         ...         ...   
 230  136.720856  138.283875  139.506714  140.443329  141.178650  141.757721   
 231  136.011871  137.564331  138.768677  139.701904  140.425598  140.997589   
 232  135.071625  136.605316  137.786896  138.712921  139.425415  139.994629   
 233  133.805878  135.308197  136.476959  137.382812  138.093475  138.645081   
 234  132.131989  133.611115  134.750488  135.643646  136.326447  136.874756   
 
             6           7

In [ ]:
fig,ax = plt.subplots(4,len(Rodinia))
for bm,index in zip(Rodinia,range(0,len(Rodinia))):
    
    #Row 1: Heatmap of real gpu
    hw_base = pd.read_csv(hwDir + bm + '_heatmap.csv',header=None)
    hwtemp = (hw_base-hw_base.min().min()) / (hw_base.max().max()-hw_base.min().min())
    im = ax[index].imshow(hwtemp, interpolation='nearest',cmap=plt.get_cmap('inferno'))
    ax[0][index].set_title(bm,fontsize=14)
    ax[0][index].set_yticks([ ])
    ax[0][index].set_xticks([ ])
    ax[0][index].set_ylabel('Baseline',fontsize=16)
    im.set_clim(0,1)
    
    if (index % len(Rodinia)):
       ax.cbar_axes[0].colorbar(im)
    
    #Row 2: Difference Map of real GPU
    #comparing differences between benchmarks hardware thermals 
    hw_diff = hw_base.copy()
    hw_diff -= (hw_avg * hw_diff.max().max())
    
    avg = (hw_diff.min().min() + hw_diff.max().max()) / 2
    ran = (hw_diff.max().max() - hw_diff.min().min()) / 2
    hw_diff = (hw_diff - avg) / ran
    ax[(len(Rodinia)*1) + index].set_yticks([ ])
    ax[(len(Rodinia)*1) + index].set_xticks([ ])
    ax(len(Rodinia)*1) + index].set_ylabel('HW Diff',fontsize=14)
    im = ax[(len(Rodinia)*1)+index].imshow(hw_diff, interpolation='nearest',cmap=plt.get_cmap('seismic'))
    im.set_clim(-1,1)
    if (index % len(Rodinia)):
       ax.cbar_axes[1].colorbar(im)
    
    #Row 3: heatmap of gpgpusim
    #sim = pd.read_csv(steadyDir + bm + '_' + flp + '.grid.steady',header=None)
    sim = pd.read_csv(simDir + bm + '.grid.steady',header=None)
    sim = pd.DataFrame(np.array(Image.fromarray(np.flipud(sim.to_numpy())).resize((252,234))))
    sim -= 273.15 #Kelvin to Celsius
    ax[(len(Rodinia)*2) + index].set_yticks([ ])
    ax[(len(Rodinia)*2) + index].set_xticks([ ])
    ax[(len(Rodinia)*2) + index].set_ylabel('GPUCalorie',fontsize=14)
    simtemp = (sim-sim.min().min()) / (sim.max().max()-sim.min().min())
    ax[(len(Rodinia)*2)+index].imshow(simtemp, interpolation='nearest',cmap=plt.get_cmap('inferno')) 
    im.set_clim(0,1)
    """
    for key in chip:
        x = int(chip[key][2]//xsize)
        width = int((chip[key][0]//xsize))
        y = int(chip[key][3]//ysize)
        length = int((chip[key][1]//ysize))
        rect = patches.Rectangle((x, y), width, length, linewidth=1, edgecolor='r', facecolor='none')
        grid[(len(Rodinia)*2)+index].add_patch(rect)

    """
    if (index % len(Rodinia)):
       grid.cbar_axes[2].colorbar(im)
    
    #Row 4: Difference map ofgpgpusim
    sim_diff = sim.copy()
    sim_diff -= (sim_avg * sim_diff.max().max())
    avg = (sim_diff.min().min() + sim_diff.max().max()) / 2
    ran = (sim_diff.max().max() - sim_diff.min().min()) / 2
    sim_diff = (sim_diff - avg) / ran
    grid[(len(Rodinia)*3) + index].set_yticks([ ])
    grid[(len(Rodinia)*3) + index].set_xticks([ 
    grid[(len(Rodinia)*3) + index].set_xticks([ ])
    grid[(len(Rodinia)*3) + index].set_ylabel('GPUCalorie Diff',fontsize=14)
    im = grid[(len(Rodinia)*3)+index].imshow(sim_diff, interpolation='nearest',cmap=plt.get_cmap('seismic'))
    #im = grid[(len(Rodinia)*3)+index].contourf(X,Y,sim_diff,20,cmap='seismic',origin='upper')
    im.set_clim(-1,1)

    if (index % len(Rodinia)):
       grid.cbar_axes[3].colorbar(im)
    
    
    #diff = np.linalg.norm((hw_diff - sim_diff).to_numpy(),ord=2) / np.linalg.norm(hw_diff.to_numpy(),ord=2)
    diff = np.linalg.norm((hw_base - sim).to_numpy(),ord=2) / np.linalg.norm(hw_base.to_numpy(),ord=2)
    #diff = abs(hw_base.mean().mean() - sim.mean().mean()) / ((hw_base.mean().mean() + sim.mean().mean())/ 2)
    print(bm+','+str(diff*100))

    #end for flpt
#end for bm


#fig.savefig("floorplan_results.pdf",bbox_inches='tight')
plt.show()])
    grid[(len(Rodinia)*3) + index].set_ylabel('GPUCalorie Diff',fontsize=14)
    im = grid[(len(Rodinia)*3)+index].imshow(sim_diff, interpolation='nearest',cmap=plt.get_cmap('seismic'))
    #im = grid[(len(Rodinia)*3)+index].contourf(X,Y,sim_diff,20,cmap='seismic',origin='upper')
    im.set_clim(-1,1)

    if (index % len(Rodinia)):
       grid.cbar_axes[3].colorbar(im)
    
    
    #diff = np.linalg.norm((hw_diff - sim_diff).to_numpy(),ord=2) / np.linalg.norm(hw_diff.to_numpy(),ord=2)
    diff = np.linalg.norm((hw_base - sim).to_numpy(),ord=2) / np.linalg.norm(hw_base.to_numpy(),ord=2)
    #diff = abs(hw_base.mean().mean() - sim.mean().mean()) / ((hw_base.mean().mean() + sim.mean().mean())/ 2)
    print(bm+','+str(diff*100))

    #end for flpt
#end for bm


#fig.savefig("floorplan_results.pdf",bbox_inches='tight')
plt.show()